In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import ncps 
from ncps.torch import LTC
from ncps.torch import CfC
from ncps.wirings import AutoNCP
import pytorch_lightning as pl
import torch.utils.data as data
import seaborn as sns
from ncps.wirings import NCP
import tensorflow as tf
sns.set()

2025-03-16 16:24:08.585503: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 16:24:09.164695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-16 16:24:10.729179: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
initial_point = np.array([10.61098536,  5.87720862, 34.48052002])
params = [10, 28, 8/3]
dt = 0.001

num_samples = [100_000, 20_000, 5_000]

def dpdt(point, params=params): #position

    x,y,z = point
    sig, rho, beta = params
    
    new_x = y*dt*sig + x*(1-dt*sig)
    new_y = x*dt*(rho-z) + y*(1-dt)
    new_z = x*y*dt + z*(1-dt*beta)
    return np.array([new_x, new_y, new_z])

def make_lorenz_rollout(num_samples):
    positions = []
    positions.append(initial_point)

    # your dataset
    for _ in range(num_samples):
        positions.append(dpdt(positions[-1]))

    positions = np.stack(positions)

    labels = np.sum(np.sqrt(np.square(positions[1:num_samples+1] - positions[:num_samples])), axis=1)

    return positions, labels

# Load Models

### NCPS

In [3]:
test_samples = num_samples[0]

In [4]:
out_features = 3
in_features = 3

#wiring = AutoNCP(6, out_features)  
wiring = NCP(inter_neurons=6, command_neurons=5, motor_neurons=out_features,
             sensory_fanout=4, inter_fanout=2, recurrent_command_synapses=4, motor_fanin=2)

num_s = str(test_samples)
PATH = f"models/CFC_state_dict_N{num_s}"

with open(PATH + '/adj_mat.npy', 'rb') as f:
    adj = np.load(f)

with open(PATH + '/sens_mat.npy', 'rb') as f:
    sens = np.load(f)

wiring.adjacency_matrix = adj
wiring.sensory_adjacency_matrix = sens

ncps = CfC(in_features, wiring, batch_first=True, return_sequences=False) # change to cfc, what could go wrong?
ncps.load_state_dict(torch.load(PATH+ "/weights", weights_only=True))




FileNotFoundError: [Errno 2] No such file or directory: 'models/CFC_state_dict_N100000/adj_mat.npy'

In [ ]:
out_features = 3
in_features = 3

#wiring = AutoNCP(6, out_features)  
wiring = NCP(inter_neurons=6, command_neurons=5, motor_neurons=out_features,
             sensory_fanout=4, inter_fanout=2, recurrent_command_synapses=4, motor_fanin=2)

ncps = CfC(in_features, wiring, batch_first=True, return_sequences=False) 



In [ ]:
from tqdm import tqdm
def parse_data(positions, traj_len, batch_size):
    """
    positions: numpy array of shape (num_steps, input_dim)
    traj_len : length of the trajectory for each (x,y) pair
    batch_size: how many samples per mini-batch

    Returns:
       data_x of shape (num_batches, batch_size, input_dim)
       data_y of shape (num_batches, batch_size, traj_len, input_dim)
    """
    print("Positions shape:", positions.shape)

    # Normalize by p_max
    p_max = np.max(positions)
    p_min = np.min(positions)
    positions = positions / p_max

    # Collect (x, y) pairs
    x = []
    y = []
    for i in range(positions.shape[0] - traj_len):
        x.append(positions[i, :])
        y.append(positions[i + 1 : i + 1 + traj_len, :])

    data_x = torch.FloatTensor(np.stack(x))  # (N, input_dim)
    data_y = torch.FloatTensor(np.stack(y))  # (N, traj_len, input_dim)

    # Truncate any leftover to make it divisible by batch_size
    num_samples = data_x.shape[0]
    remainder = num_samples % batch_size
    if remainder != 0:
        data_x = data_x[:-remainder]
        data_y = data_y[:-remainder]

    # Now reshape to (num_batches, batch_size, ...)
    num_batches = data_x.shape[0] // batch_size
    data_x = data_x.view(num_batches, batch_size, -1)
    # data_y has shape (N, traj_len, input_dim)
    data_y = data_y.view(num_batches, batch_size, data_y.shape[1], data_y.shape[2])

    return data_x, data_y


    
def training_loop(model, positions, epochs, traj_len):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    losses = []
    loss_fn = nn.MSELoss()
    
    # Get data
    x, y = parse_data(positions, traj_len = 1, batch_size=16)

    for epoch in tqdm(range(epochs)):
        for i in range(x.shape[0]):

            inputs = x[i]
            labels = y[i]
            print(inputs.shape, labels.shape)
            noise = torch.randn(inputs.shape) * 0.005 # Add noise to states

            inputs = inputs + noise

            hx = None
            outputs = []

            for i in range(traj_len):
                inputs, hx = model(inputs, hx)
                inputs = inputs.unsqueeze(0)
                print(inputs)
                outputs.append(inputs)

            outputs = torch.stack(outputs)
            
            model_loss = loss_fn(outputs, labels)
                
            optimizer.zero_grad()
            model_loss.backward()
            optimizer.step()

            losses.append(model_loss.item())

    return losses
    

traj_len = 2
positions, _ = make_lorenz_rollout(test_samples)
x, y = parse_data(positions, traj_len, 16)
print(x.shape, " ", y.shape)
#print(x[0, :, :], " \n", y[0, :, :])

losses = training_loop(ncps, positions, 10, traj_len)

In [ ]:
plt.plot(losses)
plt.yscale("log")

### Visualize

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Obtain the graph from the wiring.
G = wiring.get_graph()

# Separate nodes by type.
sensory_nodes = [node for node, data in G.nodes(data=True) if data.get("neuron_type") == "sensory"]
inter_nodes   = [node for node, data in G.nodes(data=True) if data.get("neuron_type") == "inter"]
command_nodes = [node for node, data in G.nodes(data=True) if data.get("neuron_type") == "command"]
motor_nodes   = [node for node, data in G.nodes(data=True) if data.get("neuron_type") == "motor"]

# Assign fixed x-coordinates for a feed-forward layout:
# x = 0 for sensory, 2 for inter, 4 for command, and 6 for motor.
pos = {}
layers = [
    ("sensory", sensory_nodes),
    ("inter", inter_nodes),
    ("command", command_nodes),
    ("motor", motor_nodes)
]
x_spacing = 2.0  # horizontal spacing between layers
y_spacing = 1.0  # vertical spacing within a layer

for layer_index, (layer_name, nodes) in enumerate(layers):
    x = layer_index * x_spacing
    n = len(nodes)
    # Evenly space nodes vertically and center them if there's more than one.
    for i, node in enumerate(sorted(nodes)):
        y = (i - (n - 1) / 2) * y_spacing if n > 1 else 0
        pos[node] = (x, y)

# Separate edges into non-recurrent and recurrent (command -> command) edges.
non_recurrent_edges = []
recurrent_edges = []
for u, v, d in G.edges(data=True):
    # If both source and target are command neurons, mark as recurrent.
    if G.nodes[u]["neuron_type"] == "command" and G.nodes[v]["neuron_type"] == "command":
        recurrent_edges.append((u, v))
    else:
        non_recurrent_edges.append((u, v))

plt.figure(figsize=(10, 6))

# Draw nodes with different colors for each layer.
nx.draw_networkx_nodes(G, pos, nodelist=sensory_nodes, node_color='tab:olive', label='Sensory')
nx.draw_networkx_nodes(G, pos, nodelist=inter_nodes,   node_color='tab:blue',   label='Inter')
nx.draw_networkx_nodes(G, pos, nodelist=command_nodes, node_color='tab:orange', label='Command')
nx.draw_networkx_nodes(G, pos, nodelist=motor_nodes,   node_color='tab:green',  label='Motor')

# Draw non-recurrent edges with a gentle curve.
nx.draw_networkx_edges(
    G, pos,
    edgelist=non_recurrent_edges,
    arrows=True,
    arrowstyle='->',
    connectionstyle='arc3, rad=0.2',  # gentle curve
    alpha=0.7
)

# Draw recurrent edges with a more pronounced curve and a different color.
nx.draw_networkx_edges(
    G, pos,
    edgelist=recurrent_edges,
    arrows=True,
    arrowstyle='->',
    connectionstyle='arc3, rad=0.4',  # increased curvature
    edge_color='red',
    alpha=0.7
)

# Draw node labels.
nx.draw_networkx_labels(G, pos, font_size=8)

plt.axis('off')
plt.title("Lorenz NCP")
plt.legend()
plt.tight_layout()
plt.show()


### Res

In [ ]:
def build_world_model_residual():
    xin_s = tf.keras.layers.Input((3,))

    xs = tf.keras.layers.Dense(32, activation='swish')(xin_s)
    xs = tf.keras.layers.Dense(32, activation='swish')(xs)
    xs = tf.keras.layers.Dense(3)(xs)

    xout = tf.keras.layers.Add()([xs,xin_s])
    return tf.keras.models.Model(xin_s, xout)

build_world_model_residual().summary()

res = build_world_model_residual()
res.compile(loss='mse',optimizer='adam')

PATH = f"models/RESNET_state_dict_N{num_s}.weights.h5"
res.load_weights(PATH)


### RNN

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, obs_space_size, action_space_size, hidden_size):
        super().__init__()
        self.l0 = nn.Sequential(
            nn.Linear(obs_space_size + action_space_size, 32),
            nn.ReLU()
        )
        
        self.rnn = nn.RNN(32, hidden_size, batch_first=True)

        self.l1 = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Linear(32, obs_space_size),
        )
        self.hidden_size = hidden_size

    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(1, x.size(0), self.hidden_size)

        out = self.l0(x)
        out, ht = self.rnn(out, h0)
        out = self.l1(out)
        out = out[:, -1, :]
        return out, ht

obs_space_size = 3
action_space_size = 0
hidden_size = 16
rnn = SimpleRNN(obs_space_size, action_space_size, hidden_size)

num_s = str(test_samples)
PATH = f"models/RNN_state_dict_N{num_s}"
rnn.load_state_dict(torch.load(PATH, weights_only=True))

# Liquid

In [ ]:
liquid = LTC(in_features, out_features, batch_first=True, return_sequences=False)
PATH = f"models/LTC_N{num_s}"
liquid.load_state_dict(torch.load(PATH, weights_only=True))

# Cool Graphs

In [ ]:
test_samples = num_samples[2]
positions, _ = make_lorenz_rollout(test_samples)

print(positions.shape)

p_max = np.max(positions)
p_min = np.min(positions)

# Add noise
#positions = positions+np.random.randn(test_samples + 1,3) * 0.1

# Scale
positions = positions/p_max # (normalize to between 1 some negative number that won't be 1) 

x = []
y = []
traj_len = 5
for i in range(test_samples - traj_len):
    x.append(positions[i:i+traj_len])
    y.append(positions[i+traj_len:i+traj_len+1])

#print(x, y[0])
#print(" ")
#print(x, y[1])
data_x = np.stack(x)
data_y = np.stack(y)

t_inputs = torch.FloatTensor(data_x)
print(t_inputs.shape)


In [ ]:
# now we have to make them into batches of sequence lengths

with torch.no_grad():
    #rnn_pred = rnn.forward(t_inputs)[0].numpy()
    ncps_pred = ncps(t_inputs)[0].numpy()

#res_pred = np.array(res.predict(positions[0:-1]))
#res_pred.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import torch

# Example data
# t_inputs = torch.FloatTensor(inputs).unsqueeze(0)
# with torch.no_grad():
#     liq_pred = liquid(t_inputs)[0].numpy()
#     rnn_pred = rnn.forward(t_inputs)[0].numpy()
# res_pred = np.array(res.predict(inputs))

# For demonstration, assume these are each [T, 3] arrays:
# liq_pred, rnn_pred, res_pred, inputs

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the real data (inputs) as a thick line with no transparency
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], linewidth=5, alpha=1.0, label='Real', color='black')

# Plot predicted trajectories with thinner lines and partial transparency
ax.plot(ncps_pred[:, 0], ncps_pred[:, 1], ncps_pred[:, 2],
         linewidth=2, alpha=1, label='NCPS Prediction')
ax.plot(rnn_pred[:, 0], rnn_pred[:, 1], rnn_pred[:, 2],
        linewidth=2, alpha=1, label='RNN Prediction')
ax.plot(res_pred[:, 0], res_pred[:, 1], res_pred[:, 2],
         linewidth=2, alpha=1, label='Res Prediction')

# Create small shaded polygons connecting each real point to the corresponding
# predicted points. This can be visually heavy if T is large, so consider
# downsampling if needed.


# Label the axes and set a title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Trajectories Comparison')

ax.legend()
plt.show()


In [ ]:
test_samples = num_samples[2]
initial_point = np.array([-0.510, -0.4218712, -0.23523])
dt = 0.003
positions, _ = make_lorenz_rollout(test_samples)

print(positions.shape)

p_max = np.max(positions)
p_min = np.min(positions)

# Add noise
positions = positions+np.random.randn(test_samples + 1,3) * 0.1

# Scale
positions = positions/p_max # (normalize to between 1 some negative number that won't be 1) 

x = []
y = []
traj_len = 5
for i in range(test_samples - traj_len):
    x.append(positions[i:i+traj_len])
    y.append(positions[i+traj_len:i+traj_len+1])

#print(x, y[0])
#print(" ")
#print(x, y[1])
data_x = np.stack(x)
data_y = np.stack(y)

inputs = positions[0:-1]
t_inputs = torch.FloatTensor(data_x)
print(t_inputs.shape)
print(inputs.shape)

# now we have to make them into batches of sequence lengths

with torch.no_grad():
    liq_pred = liquid(t_inputs)[0].numpy()
    ncps_pred = liquid(t_inputs)[0].numpy()
    rnn_pred = rnn.forward(t_inputs)[0].numpy()

res_pred = np.array(res.predict(inputs))
res_pred.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import torch

# Example data
# t_inputs = torch.FloatTensor(inputs).unsqueeze(0)
# with torch.no_grad():
#     liq_pred = liquid(t_inputs)[0].numpy()
#     rnn_pred = rnn.forward(t_inputs)[0].numpy()
# res_pred = np.array(res.predict(inputs))

# For demonstration, assume these are each [T, 3] arrays:
# liq_pred, rnn_pred, res_pred, inputs

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the real data (inputs) as a thick line with no transparency
ax.plot(inputs[:, 0], inputs[:, 1], inputs[:, 2],
     linewidth=5, alpha=1.0, label='Real', color='black')

# Plot predicted trajectories with thinner lines and partial transparency
ax.plot(ncps_pred[:, 0], ncps_pred[:, 1], ncps_pred[:, 2],
         linewidth=2, alpha=1, label='NCPS Prediction')
#ax.plot(liq_pred[:, 0], liq_pred[:, 1], liq_pred[:, 2],
#        linewidth=2, alpha=1, label='Liquid Prediction')
ax.plot(rnn_pred[:, 0], rnn_pred[:, 1], rnn_pred[:, 2],
   linewidth=2, alpha=1, label='RNN Prediction')
ax.plot(res_pred[:, 0], res_pred[:, 1], res_pred[:, 2],
      linewidth=2, alpha=1, label='Res Prediction')

# Create small shaded polygons connecting each real point to the corresponding
# predicted points. This can be visually heavy if T is large, so consider
# downsampling if needed.

# Label the axes and set a title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('OOD 3D Trajectories - One Step Comparison')

ax.legend()
plt.show()


# Multi-Step Rollouts

In [ ]:
l_rolling_inputs = r_rolling_inputs = n_rolling_inputs = t_inputs[0]
print(t_inputs[0])
window = inputs.shape[1]

r_rolling_inputs = r_rolling_inputs.unsqueeze(axis=0)

#print(r_rolling_inputs.shape)

liquid_predictions = []
rnn_predictions = []
ncps_predictions = []

num_predictions = 10_000

for i in range(num_predictions):
    with torch.no_grad():
        l_prediction = liquid(l_rolling_inputs)[0].unsqueeze(0)
        r_prediction = rnn(r_rolling_inputs)[0].unsqueeze(0)
        n_prediction = ncps(n_rolling_inputs)[0].unsqueeze(0)


    liquid_predictions.append(l_prediction)
    rnn_predictions.append(r_prediction)
    ncps_predictions.append(n_prediction)
    
    #prediction = torch.FloatTensor(np.expand_dims(prediction, axis=0))
    #print(r_prediction.shape)
    #print(r_rolling_inputs.shape)
    
    l_rolling_inputs = torch.concatenate((l_rolling_inputs[1:window+1], l_prediction))
    r_rolling_inputs = torch.concatenate((r_rolling_inputs[:, 1:window+1, :], r_prediction), axis=1)
    n_rolling_inputs = torch.concatenate((n_rolling_inputs[1:window+1], n_prediction))


    print(n_rolling_inputs, " ", prediction)


#predictions.shape


In [ ]:
liquid_predictions = torch.stack(liquid_predictions).squeeze(1).numpy()
rnn_predictions = torch.stack(rnn_predictions).squeeze(1).numpy()
ncps_predictions = torch.stack(ncps_predictions).squeeze(1).numpy()
liquid_predictions.shape

In [5]:
liquid_predictions.shape
rnn_predictions.shape
ncps_predictions.shape

NameError: name 'liquid_predictions' is not defined

In [39]:
liquid_predictions[2, :]

NameError: name 'liquid_predictions' is not defined

In [40]:
start = inputs[3:4, :]
start

NameError: name 'inputs' is not defined

In [ ]:
res_prediction = []
#start = res(t_inputs[0, -1, :].unsqueeze(0).numpy())
print(start)
res_prediction.append(np.array(res(start)))
for i in range(num_predictions-1):
    # random action
    _state = np.array(res(res_prediction[-1]))
    #print(_state.shape)
    # pend length
    res_prediction.append(_state)

In [41]:
res_prediction = np.stack(res_prediction)
res_prediction.shape

ValueError: need at least one array to stack

In [ ]:

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the real data (inputs) as a thick line with no transparency
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2],
     linewidth=1, alpha=0.2, label='Real', color='black')

#Plot predicted trajectories with thinner lines and partial transparency
ax.plot(ncps_predictions[:, 0], ncps_predictions[:, 1], ncps_predictions[:, 2],
        linewidth=2, alpha=1, label='NCPS Prediction')
ax.plot(liquid_predictions[:, 0], liquid_predictions[:, 1], liquid_predictions[:, 2],
        linewidth=2, alpha=1, label='Liquid Prediction')
ax.plot(rnn_predictions[:,0, 0], rnn_predictions[:, 0, 1], rnn_predictions[:,0, 2],
   linewidth=2, alpha=1, label='RNN Prediction')
ax.plot(res_prediction[:, 0, 0], res_prediction[:, 0, 1], res_prediction[:, 0, 2],
      linewidth=2, alpha=1, label='Res Prediction')

# Create small shaded polygons connecting each real point to the corresponding
# predicted points. This can be visually heavy if T is large, so consider
# downsampling if needed.

# Label the axes and set a title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('100k Step Prediction')

ax.legend()
plt.show()


# Decoherence

In [18]:
# We set an arbitrary X, an upper bound for max euclidean distance 
initial_point = np.array([10.61098536,  5.87720862, 34.48052002])
params = [10, 28, 8/3]
dt = 0.001

positions, labels = make_lorenz_rollout(test_samples)
positions /= np.max(positions)
labels /= np.max(positions)
X = np.max(labels) + 1/4 * np.max(labels)
X

0.5351068617084742

In [19]:
np.random.seed(101)
start_positions = np.random.randint(5, test_samples - 100, size=50)

In [20]:


# for residuals
test_positions_rec = []
real_positions = []
for i in range(5):
    test_positions_rec.append(positions[start_positions-5+i, :])

for i in range(100):
    real_positions.append(positions[start_positions+i, :])
    
test_positions_rec = np.stack(test_positions_rec)
real_positions = np.stack(real_positions).swapaxes(0, 1)
print(real_positions.shape)

test_positions_residual = test_positions_rec[-1, :, :]
test_positions_rec = test_positions_rec.swapaxes(0, 1)
test_positions_rec.shape

#print(test_positions_rec[0, :, :], real_positions[0, 0, :])
#print(positions[start_positions[0]])


(50, 100, 3)


(50, 5, 3)

In [21]:
l_rolling_inputs = r_rolling_inputs = n_rolling_inputs = torch.FloatTensor(test_positions_rec)
window = inputs.shape[1]

#print(r_rolling_inputs.shape)

liquid_predictions = []
rnn_predictions = []
ncps_predictions = []

num_predictions = 100

for i in range(num_predictions):
    with torch.no_grad():
        l_prediction = liquid(l_rolling_inputs)[0].unsqueeze(0).swapaxes(0, 1)
        r_prediction = rnn(r_rolling_inputs)[0].unsqueeze(0).swapaxes(0, 1)
        n_prediction = ncps(n_rolling_inputs)[0].unsqueeze(0).swapaxes(0, 1)


    #print(l_prediction.shape, " ", r_prediction.shape)
    liquid_predictions.append(l_prediction)
    rnn_predictions.append(r_prediction)
    ncps_predictions.append(n_prediction)
    #print(l_rolling_inputs.shape, " ", r_rolling_inputs.shape)
    
    #prediction = torch.FloatTensor(np.expand_dims(prediction, axis=0))
    #print(r_prediction.shape)
    #print(r_rolling_inputs.shape)

    l_rolling_inputs = torch.concatenate((l_rolling_inputs[:, 1:window+2, :], l_prediction), axis=1)
    r_rolling_inputs = torch.concatenate((r_rolling_inputs[:, 1:window+2, :], r_prediction), axis=1)
    n_rolling_inputs = torch.concatenate((n_rolling_inputs[:, 1:window+2, :], n_prediction), axis=1)


    #print(rolling_inputs, " ", prediction)


NameError: name 'inputs' is not defined

In [22]:
liquid_predictions = torch.stack(liquid_predictions).squeeze(2).numpy().swapaxes(0, 1)
rnn_predictions = torch.stack(rnn_predictions).squeeze(2).numpy().swapaxes(0, 1)
ncps_predictions = torch.stack(ncps_predictions).squeeze(2).numpy().swapaxes(0, 1)
liquid_predictions.shape

NameError: name 'liquid_predictions' is not defined

In [23]:
liquid_res = liquid_predictions - real_positions
liquid_predictions
print(liquid_predictions[0, 0, :], real_positions[0, 0, :] , liquid_res[0, 0, :])
liquid_res_summed = np.sum(np.abs(liquid_res[0]), axis=-1)
liquid_res_summed

NameError: name 'liquid_predictions' is not defined

In [24]:
liquid_res = np.sum(np.sqrt(np.square(liquid_predictions - real_positions)), axis=-1)
rnn_res = np.sum(np.sqrt(np.square(rnn_predictions - real_positions)), axis=-1)
ncps_res = np.sum(np.sqrt(np.square(ncps_predictions - real_positions)), axis=-1)

NameError: name 'liquid_predictions' is not defined

In [25]:
liquid_res

NameError: name 'liquid_res' is not defined

In [26]:
liquid_res.shape

NameError: name 'liquid_res' is not defined

In [27]:
liquid_mask = (liquid_res > X)
res_mask = (ncps_res > X)

NameError: name 'liquid_res' is not defined

In [ ]:
res_mask

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
